In [191]:
import pandas as pd
import numpy as np
from pathlib import Path

In [193]:
# load the data set
file_path = Path("..") / "data" / "raw" / "2019-population_census.xlsx"

In [195]:
# Read the excel file
df = pd.read_excel(file_path, sheet_name='Nairobi County')

In [197]:
# Inspecting data set
df.head(10)

,Unnamed: 0,Unnamed: 1,Total,Sex*,Unnamed: 4,Unnamed: 5,Households,Unnamed: 7,Land Area,Density
0,NaN,NaN,NaN,Male,Female,Total,Conventional,Group Quarters,Sq. Km,Persons per Sq. Km
1,NAIROBI CITY,County,4397073.0,2192452,2204376,1506888,1494676,12212,703.87005,6247
2,DAGORETTI,Sub-county,434208.0,217651,216526,155089,154949,140,29.12662,14908
3,KAWANGWARE,Division,291565.0,146666,144879,105258,105217,41,11.0578,26367
4,GATINA,Location,63560.0,33169,30389,24005,23996,9,1.48582,42778
5,CONGO,Sub-location,31914.0,16356,15557,12160,12160,-,0.70198,45463
6,GATINA,Sub-location,31646.0,16813,14832,11845,11836,9,0.78384,40373
7,KAWANGWARE,Location,91487.0,45682,45801,33008,32981,27,2.43753,37533
8,KABIRO,Sub-location,36228.0,18160,18068,13328,13317,11,0.77465,46767
9,KAWANGWARE,Sub-location,30587.0,14581,16002,10893,10877,16,0.9102,33605


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  263 non-null    object 
 1   Unnamed: 1  262 non-null    object 
 2   Total       261 non-null    float64
 3   Sex*        263 non-null    object 
 4   Unnamed: 4  263 non-null    object 
 5   Unnamed: 5  263 non-null    object 
 6   Households  263 non-null    object 
 7   Unnamed: 7  263 non-null    object 
 8   Land Area   263 non-null    object 
 9   Density     263 non-null    object 
dtypes: float64(1), object(9)
memory usage: 20.8+ KB


In [201]:
# Remove unnamed columns that are all NaN
cols_to_keep = []
for col in df.columns:
    if not col.startswith('Unnamed') or not df[col].isna().all():
        cols_to_keep.append(col)
    
df_cleaned = df[cols_to_keep].copy()

In [203]:
df_cleaned.head(5)

,Unnamed: 0,Unnamed: 1,Total,Sex*,Unnamed: 4,Unnamed: 5,Households,Unnamed: 7,Land Area,Density
0,NaN,NaN,NaN,Male,Female,Total,Conventional,Group Quarters,Sq. Km,Persons per Sq. Km
1,NAIROBI CITY,County,4397073.0,2192452,2204376,1506888,1494676,12212,703.87005,6247
2,DAGORETTI,Sub-county,434208.0,217651,216526,155089,154949,140,29.12662,14908
3,KAWANGWARE,Division,291565.0,146666,144879,105258,105217,41,11.0578,26367
4,GATINA,Location,63560.0,33169,30389,24005,23996,9,1.48582,42778


In [205]:
#Remove rows that are entirely NaN
df_cleaned = df_cleaned.dropna(how='all')

In [207]:
# Rename columns appropriately
if len(df_cleaned.columns) >= 9:
    new_column_names = {
            df_cleaned.columns[0]: 'Administrative_Unit',
            df_cleaned.columns[1]: 'Administrative_level',
            df_cleaned.columns[2]: 'Total_Population', 
            df_cleaned.columns[3]: 'Male_Population',
            df_cleaned.columns[4]: 'Female_Population',
            df_cleaned.columns[5]: 'Total_Households',
            df_cleaned.columns[6]: 'Conventional_Households',
            df_cleaned.columns[7]: 'Group_Quarters',
            df_cleaned.columns[8]: 'Land_Area_SqKm',
            df_cleaned.columns[9]: 'Population_Density'
        }
df_cleaned = df_cleaned.rename(columns=new_column_names)

# remove the first row
df_cleaned = df_cleaned.drop(index = 0)

In [209]:
df_cleaned.head(5)

,Administrative_Unit,Administrative_level,Total_Population,Male_Population,Female_Population,Total_Households,Conventional_Households,Group_Quarters,Land_Area_SqKm,Population_Density
1,NAIROBI CITY,County,4397073.0,2192452,2204376,1506888,1494676,12212,703.87005,6247
2,DAGORETTI,Sub-county,434208.0,217651,216526,155089,154949,140,29.12662,14908
3,KAWANGWARE,Division,291565.0,146666,144879,105258,105217,41,11.0578,26367
4,GATINA,Location,63560.0,33169,30389,24005,23996,9,1.48582,42778
5,CONGO,Sub-location,31914.0,16356,15557,12160,12160,-,0.70198,45463


In [211]:
def administration_level_summary(df):
    """Summary of the administration level of the cleaned Nairobi data"""
    print("\n" + "="*60)
    print("NAIROBI County Administration levels")
    print("="*60)

     # Strip whitespace and standardize case
    df_cleaned['Administrative_level'] = df_cleaned['Administrative_level'].str.strip().str.title()
    
    #Summary by administrative level
    print("\n Summary by Administrative Level:")
    level_summary = df.groupby('Administrative_level').agg({
        'Total_Population': ['count', 'sum'],
        'Total_Households': 'sum',
        'Land_Area_SqKm': 'sum'
    }).round(2)
    level_summary.columns = ['Count', 'Total_Population', 'Total_Households', 'Total_Land_Area']
    print(level_summary)

In [213]:
# Call the function
administration_level_summary(df_cleaned)


NAIROBI County Administration levels

 Summary by Administrative Level:
                      Count  Total_Population Total_Households Total_Land_Area
Administrative_level                                                          
County                    1         4397073.0          1506888       703.87005
Division                 30         4397073.0          1506888       703.87006
Location                 77         4397073.0          1506888       703.87006
Sub-County               11         4397073.0          1506888       703.87006
Sub-Location            142         4383489.0          1506888       703.87009


In [215]:
# Save the cleaned dataframe
def save_cleaned_data(df):
    """Save the cleaned DataFrame to fixed Excel and CSV paths"""

    # Define the base output path (without extension)
    output_base = Path("..") / "data" / "clean" / "nairobi_census_2019_cleaned"

    # Ensure the parent directory exists
    output_base.parent.mkdir(parents=True, exist_ok=True)

    # Save to Excel
    df.to_excel(output_base.with_suffix('.xlsx'), index=False)

    # Save to CSV
    df.to_csv(output_base.with_suffix('.csv'), index=False)

    # Print confirmation
    print("\nCleaned data saved to:")
    print(f"  - {output_base.with_suffix('.xlsx')}")
    print(f"  - {output_base.with_suffix('.csv')}")

    return df

In [217]:
# Calling the function
if __name__ == "__main__":
    try:
        save_cleaned_data(df_cleaned)
    except Exception as e:
        print(f"Error processing file: {e}")


Cleaned data saved to:
  - ..\data\clean\nairobi_census_2019_cleaned.xlsx
  - ..\data\clean\nairobi_census_2019_cleaned.csv
